# Chapter 3 Exercise 3: Uplift 模型评估 - 如何知道模型好不好?

## 学习目标

1. 理解 Qini 曲线的原理和计算方法
2. 掌握 AUUC (Area Under Uplift Curve) 指标
3. 学会使用累积增益分析
4. 理解最优干预比例的确定方法

---

## 为什么需要特殊的评估方法?

### 传统分类问题 vs Uplift 问题

| 问题类型 | 目标 | 可用信息 | 评估方法 |
|---------|------|---------|--------|
| 分类问题 | 预测 Y | 真实 Y 可观测 | AUC, Precision, Recall |
| Uplift 问题 | 预测 τ = Y(1) - Y(0) | 只能观测 Y(T)，无法观测反事实 | ??? |

**核心挑战**：我们无法直接观测到真实的个体处理效应！

那怎么办？我们需要一些 **巧妙的间接方法**。

## 核心思想：排序能力

虽然我们无法验证预测的绝对值，但可以验证 **排序能力**：

> 如果模型是好的，那么按预测 Uplift 排序后，排在前面的人应该有更高的真实 Uplift！

### 类比：选拔运动员

假设你是教练，想选出最有潜力的运动员参加特训：

- **好的选拔方法**：选出的人特训后提升最大
- **差的选拔方法**：随机选人，或选了本来就很强的人

Qini 曲线就是用来衡量 "选拔能力" 的！

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, List

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

np.random.seed(42)
print("环境准备完成！")

---

## Part 1: Qini 曲线

### 什么是 Qini 曲线?

Qini 曲线是 Uplift 模型版本的 ROC 曲线。它展示：**按预测 Uplift 从高到低选取用户时，累积的增量收益**。

### Qini 曲线的计算公式

对于前 k 个样本（按预测 Uplift 降序排列）：

$$Qini(k) = Y_t(k) - Y_c(k) \times \frac{n_t(k)}{n_c(k)}$$

其中：
- $Y_t(k)$: 前 k 个样本中，处理组的总转化数
- $Y_c(k)$: 前 k 个样本中，控制组的总转化数
- $n_t(k)$: 前 k 个样本中，处理组的数量
- $n_c(k)$: 前 k 个样本中，控制组的数量

### 直觉解释

$$Qini(k) = \text{处理组转化} - \text{控制组转化} \times \text{调整因子}$$

调整因子 $\frac{n_t(k)}{n_c(k)}$ 用于处理处理组和控制组人数不平衡的情况。

In [ ]:
# 练习 3.1: Qini 曲线计算

def calculate_qini_curve(
    y_true: np.ndarray,
    treatment: np.ndarray,
    uplift_score: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:
    """
    计算 Qini 曲线
    
    Qini 曲线评估模型对 uplift 的排序能力:
    - 将样本按预测 uplift 从高到低排序
    - 累积计算前 k 个样本的增益
    
    Qini(k) = Y_t(k) - Y_c(k) * (n_t(k) / n_c(k))
    
    TODO: 实现 Qini 曲线计算
    
    Args:
        y_true: 真实结果 (0/1)
        treatment: 处理状态 (0/1)
        uplift_score: 预测的 uplift 得分
    
    Returns:
        (fraction_targeted, qini_values)
        fraction_targeted: 干预比例 [0, 1]
        qini_values: 对应的 Qini 值
    """
    # TODO: 按 uplift 得分从高到低排序
    order = None  # 你的代码: np.argsort(uplift_score)[::-1]
    
    if order is None:
        return None, None
    
    y_sorted = y_true[order]
    t_sorted = treatment[order]
    
    n = len(y_true)
    
    # TODO: 计算累积统计量
    # 提示: 使用 np.cumsum
    
    # 累积处理组转化数: sum of y where t=1
    cum_t_outcomes = None  # 你的代码: np.cumsum(y_sorted * t_sorted)
    
    # 累积控制组转化数: sum of y where t=0
    cum_c_outcomes = None  # 你的代码: np.cumsum(y_sorted * (1 - t_sorted))
    
    # 累积处理组样本数
    cum_t = None  # 你的代码: np.cumsum(t_sorted)
    
    # 累积控制组样本数
    cum_c = None  # 你的代码: np.cumsum(1 - t_sorted)
    
    # TODO: 计算 Qini 值
    # Qini = cum_t_outcomes - cum_c_outcomes * (cum_t / cum_c)
    # 注意避免除零
    qini = None  # 你的代码
    
    # TODO: 计算干预比例
    fraction = None  # 你的代码: np.arange(1, n+1) / n
    
    if fraction is not None and qini is not None:
        # 添加原点 (0, 0)
        fraction = np.insert(fraction, 0, 0)
        qini = np.insert(qini, 0, 0)
    
    return fraction, qini

# 生成测试数据
np.random.seed(42)
n = 2000

# 特征
X = np.random.randn(n)

# 处理分配
T = np.random.binomial(1, 0.5, n)

# 真实 uplift (异质性)
true_uplift = 0.1 + 0.15 * (X > 0)

# 基线转化率
baseline_prob = 0.2

# 生成结果
prob = baseline_prob + true_uplift * T
Y = np.random.binomial(1, prob)

# 模拟两个模型的预测
# Model 1: 好模型 (与真实 uplift 相关)
model1_pred = true_uplift + np.random.randn(n) * 0.05

# Model 2: 差模型 (随机)
model2_pred = np.random.randn(n) * 0.1

# 测试 Qini 曲线
fraction, qini = calculate_qini_curve(Y, T, model1_pred)
if fraction is not None and qini is not None:
    print(f"计算完成: {len(fraction)} 个点")
    print(f"最大 Qini 值: {qini.max():.4f}")
else:
    print("[未完成] 请完成 calculate_qini_curve 函数")

---

## Part 2: AUUC (Area Under Uplift Curve)

### 什么是 AUUC?

AUUC 是 Qini 曲线下的面积，类似于 ROC 曲线的 AUC。

$$AUUC = \int_0^1 Qini(k) \, dk$$

### AUUC 的解释

| AUUC 值 | 含义 |
|---------|------|
| 高 | 模型能很好地识别高 Uplift 人群 |
| 接近 0 | 模型和随机选择差不多 |
| 负数 | 模型在反向选择 (选到了负 Uplift 人群) |

In [ ]:
# 练习 3.2: AUUC 计算

def calculate_auuc(
    y_true: np.ndarray,
    treatment: np.ndarray,
    uplift_score: np.ndarray
) -> float:
    """
    计算 AUUC (Area Under Uplift Curve)
    
    AUUC 是 Qini 曲线下的面积，衡量模型整体性能
    
    TODO: 实现 AUUC 计算
    提示: 可以使用 np.trapz 进行梯形积分
    
    Returns:
        AUUC value
    """
    # TODO: 先计算 Qini 曲线
    fraction, qini = calculate_qini_curve(y_true, treatment, uplift_score)
    
    if fraction is None or qini is None:
        return None
    
    # TODO: 计算曲线下面积
    auuc = None  # 你的代码: np.trapz(qini, fraction)
    
    return auuc

# 测试
auuc1 = calculate_auuc(Y, T, model1_pred)
auuc2 = calculate_auuc(Y, T, model2_pred)

if auuc1 is not None:
    print(f"Good Model AUUC: {auuc1:.4f}")
    print(f"Poor Model AUUC: {auuc2:.4f}")
    print(f"\n好模型的 AUUC 应该明显高于差模型!")
else:
    print("[未完成] 请完成 calculate_auuc 函数")

---

## Part 3: Uplift by Decile - 分桶验证

### 为什么要分桶?

Qini 曲线是一个整体视图。但在业务中，我们常常想知道：

> "预测 Uplift 最高的 10% 用户，真实 Uplift 是多少?"

分桶验证可以回答这个问题！

### 分桶验证流程

```
1. 按预测 Uplift 排序
2. 分成 10 组 (或其他数量)
3. 计算每组的真实 Uplift
4. 检查：高分组是否真的有高 Uplift?
```

In [ ]:
# 练习 3.3: Uplift by Decile

def calculate_uplift_by_decile(
    y_true: np.ndarray,
    treatment: np.ndarray,
    uplift_score: np.ndarray,
    n_deciles: int = 10
) -> pd.DataFrame:
    """
    按预测 uplift 分组，计算每组的实际 uplift
    
    这是一个重要的诊断工具:
    - 如果模型有效，高分组应该有更高的 uplift
    - 可以识别负 uplift 人群
    
    TODO: 实现分组统计
    
    Args:
        y_true: 真实结果
        treatment: 处理状态
        uplift_score: 预测得分
        n_deciles: 分组数量
    
    Returns:
        DataFrame with columns: decile, n_samples, uplift, treated_rate, control_rate
    """
    # TODO: 按 uplift_score 分成 n_deciles 组
    # 提示: 使用 pd.qcut 按分位数分组
    try:
        deciles = pd.qcut(uplift_score, n_deciles, labels=False, duplicates='drop')
    except:
        return None
    
    results = []
    
    # TODO: 对每组计算统计量
    for d in range(n_deciles):
        mask = deciles == d
        
        if mask.sum() == 0:
            continue
        
        y_sub = y_true[mask]
        t_sub = treatment[mask]
        
        # TODO: 计算该组的统计量
        n_samples = mask.sum()
        
        # 处理组和控制组转化率
        treated_mask = t_sub == 1
        control_mask = t_sub == 0
        
        if treated_mask.sum() > 0 and control_mask.sum() > 0:
            treated_rate = None  # 你的代码: y_sub[treated_mask].mean()
            control_rate = None  # 你的代码: y_sub[control_mask].mean()
            uplift = None        # 你的代码: treated_rate - control_rate
        else:
            treated_rate = 0.0
            control_rate = 0.0
            uplift = 0.0
        
        results.append({
            'decile': d + 1,  # 1-indexed, 1 = 最高分
            'n_samples': n_samples,
            'uplift': uplift,
            'treated_rate': treated_rate,
            'control_rate': control_rate
        })
    
    return pd.DataFrame(results)

# 测试
decile_df = calculate_uplift_by_decile(Y, T, model1_pred)
if decile_df is not None and len(decile_df) > 0 and decile_df['uplift'].iloc[0] is not None:
    print("Good Model - Uplift by Decile:")
    print(decile_df.to_string(index=False))
    print("\n分析: Decile 1 (最高分) 应该有最高的 Uplift!")
else:
    print("[未完成] 请完成 calculate_uplift_by_decile 函数")

---

## Part 4: 累积增益分析

### 累积增益 vs Qini 曲线

- **Qini 曲线**: 衡量排序质量
- **累积增益**: 衡量绝对价值

累积增益回答：**如果我们按模型排序干预前 k% 的人，预期能获得多少总增益？**

In [ ]:
# 练习 3.4: 累积增益分析

def calculate_cumulative_gain(
    y_true: np.ndarray,
    treatment: np.ndarray,
    uplift_score: np.ndarray,
    true_uplift: np.ndarray = None
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    计算累积增益曲线
    
    累积增益 = 按预测排序后，前 k% 人群的总 uplift
    
    TODO: 实现累积增益计算
    
    Args:
        y_true: 真实结果
        treatment: 处理状态
        uplift_score: 预测得分
        true_uplift: 真实 uplift (如果已知)
    
    Returns:
        (fraction, observed_gain, perfect_gain)
        fraction: 干预比例
        observed_gain: 模型的累积增益
        perfect_gain: 完美模型的累积增益 (如果 true_uplift 已知)
    """
    n = len(y_true)
    
    # TODO: 按预测 uplift 排序
    order = None  # 你的代码: np.argsort(uplift_score)[::-1]
    
    if order is None:
        return None, None, None
    
    y_sorted = y_true[order]
    t_sorted = treatment[order]
    
    # TODO: 计算每个样本的观测 uplift
    observed_uplift = []
    for i in range(1, n + 1):
        y_sub = y_sorted[:i]
        t_sub = t_sorted[:i]
        
        if (t_sub == 1).sum() > 0 and (t_sub == 0).sum() > 0:
            uplift = y_sub[t_sub == 1].mean() - y_sub[t_sub == 0].mean()
        else:
            uplift = 0
        
        observed_uplift.append(uplift * i)  # 累积增益
    
    fraction = np.arange(1, n + 1) / n
    observed_gain = np.array(observed_uplift)
    
    # TODO: 如果知道真实 uplift，计算完美排序的增益
    perfect_gain = None
    if true_uplift is not None:
        # 按真实 uplift 排序
        perfect_order = np.argsort(true_uplift)[::-1]
        perfect_gain = np.cumsum(true_uplift[perfect_order])
    
    return fraction, observed_gain, perfect_gain

# 测试
fraction, obs_gain, perf_gain = calculate_cumulative_gain(Y, T, model1_pred, true_uplift)
if obs_gain is not None:
    print(f"计算完成")
    print(f"观测增益 (50%): {obs_gain[int(0.5*len(obs_gain))]:.2f}")
    if perf_gain is not None:
        print(f"完美增益 (50%): {perf_gain[int(0.5*len(perf_gain))]:.2f}")
else:
    print("[未完成] 请完成 calculate_cumulative_gain 函数")

---

## Part 5: 最优干预比例

### 业务问题

在实际业务中，我们需要决定：**应该干预多少比例的用户？**

考虑因素：
- 干预成本 (如优惠券面额)
- 转化收益 (如订单金额)
- 边际效益递减

### ROI 最大化

$$ROI = \frac{\text{收入} - \text{成本}}{\text{成本}}$$

最优比例是让 ROI 最大的干预比例！

In [ ]:
# 练习 3.5: 最优干预比例

def find_optimal_targeting_fraction(
    y_true: np.ndarray,
    treatment: np.ndarray,
    uplift_score: np.ndarray,
    cost_per_treatment: float = 1.0,
    revenue_per_conversion: float = 10.0
) -> Tuple[float, float, np.ndarray, np.ndarray]:
    """
    找到最优干预比例
    
    考虑成本和收益:
    - 每次干预成本: cost_per_treatment
    - 每次转化收益: revenue_per_conversion
    
    目标: 最大化 ROI = (revenue - cost) / cost
    
    TODO: 实现最优比例搜索
    
    Returns:
        (optimal_fraction, optimal_roi, all_fractions, all_rois)
    """
    n = len(y_true)
    
    # TODO: 按 uplift 排序
    order = None  # 你的代码: np.argsort(uplift_score)[::-1]
    
    if order is None:
        return None, None, None, None
    
    y_sorted = y_true[order]
    t_sorted = treatment[order]
    
    # 测试不同的干预比例
    fractions = np.linspace(0.05, 1.0, 20)
    rois = []
    
    for frac in fractions:
        # TODO: 计算前 frac 比例人群的 ROI
        n_target = int(frac * n)
        
        if n_target == 0:
            rois.append(0)
            continue
        
        # 前 n_target 个样本
        y_top = y_sorted[:n_target]
        t_top = t_sorted[:n_target]
        
        # TODO: 估计 uplift
        if (t_top == 1).sum() > 0 and (t_top == 0).sum() > 0:
            uplift = None  # 你的代码
        else:
            uplift = 0
        
        # TODO: 计算收益和成本
        # 增量转化 = uplift * 目标人数
        incremental_conversions = None  # 你的代码: uplift * n_target
        revenue = None  # 你的代码: incremental_conversions * revenue_per_conversion
        cost = None     # 你的代码: n_target * cost_per_treatment
        
        # ROI
        if cost is not None and cost > 0:
            roi = (revenue - cost) / cost
        else:
            roi = 0
        rois.append(roi)
    
    rois = np.array(rois)
    
    # TODO: 找最优点
    optimal_idx = None  # 你的代码: np.argmax(rois)
    
    if optimal_idx is not None:
        optimal_fraction = fractions[optimal_idx]
        optimal_roi = rois[optimal_idx]
    else:
        optimal_fraction = None
        optimal_roi = None
    
    return optimal_fraction, optimal_roi, fractions, rois

# 测试
opt_frac, opt_roi, fracs, rois = find_optimal_targeting_fraction(
    Y, T, model1_pred,
    cost_per_treatment=1.0,
    revenue_per_conversion=10.0
)
if opt_frac is not None:
    print(f"最优干预比例: {opt_frac * 100:.1f}%")
    print(f"最优 ROI: {opt_roi:.2f}")
else:
    print("[未完成] 请完成 find_optimal_targeting_fraction 函数")

---

## Part 6: 可视化所有评估结果

In [ ]:
# 可视化

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 图1: Qini 曲线对比
fraction1, qini1 = calculate_qini_curve(Y, T, model1_pred)
fraction2, qini2 = calculate_qini_curve(Y, T, model2_pred)
random_qini = np.linspace(0, qini1[-1] if qini1 is not None else 0, len(fraction1) if fraction1 is not None else 2)

if fraction1 is not None:
    axes[0, 0].plot(fraction1, qini1, 'b-', linewidth=2, label='Good Model')
    axes[0, 0].plot(fraction2, qini2, 'r-', linewidth=2, label='Poor Model')
    axes[0, 0].plot(fraction1, random_qini, 'k--', linewidth=1, label='Random')
    axes[0, 0].set_xlabel('Fraction Targeted')
    axes[0, 0].set_ylabel('Qini')
    axes[0, 0].set_title('Qini Curves')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

# 图2: Uplift by Decile
if decile_df is not None and decile_df['uplift'].iloc[0] is not None:
    axes[0, 1].bar(decile_df['decile'], decile_df['uplift'], color='steelblue', alpha=0.7)
    axes[0, 1].axhline(y=0, color='red', linestyle='--', linewidth=1)
    axes[0, 1].set_xlabel('Decile (1 = Highest Score)')
    axes[0, 1].set_ylabel('Actual Uplift')
    axes[0, 1].set_title('Uplift by Decile (Good Model)')
    axes[0, 1].grid(True, alpha=0.3)

# 图3: 累积增益
if obs_gain is not None:
    axes[1, 0].plot(fraction, obs_gain, 'b-', linewidth=2, label='Model')
    if perf_gain is not None:
        axes[1, 0].plot(fraction, perf_gain, 'g--', linewidth=2, label='Perfect')
    axes[1, 0].set_xlabel('Fraction Targeted')
    axes[1, 0].set_ylabel('Cumulative Gain')
    axes[1, 0].set_title('Cumulative Gain Curve')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

# 图4: ROI vs 干预比例
if fracs is not None and rois is not None:
    axes[1, 1].plot(fracs * 100, rois, 'b-', linewidth=2)
    if opt_frac is not None:
        axes[1, 1].axvline(x=opt_frac * 100, color='red', linestyle='--', linewidth=2, 
                          label=f'Optimal: {opt_frac*100:.1f}%')
        axes[1, 1].scatter([opt_frac * 100], [opt_roi], color='red', s=100, zorder=5)
    axes[1, 1].axhline(y=0, color='gray', linestyle='-', linewidth=1)
    axes[1, 1].set_xlabel('Targeting Fraction (%)')
    axes[1, 1].set_ylabel('ROI')
    axes[1, 1].set_title('ROI vs Targeting Fraction')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Part 7: 模型对比

让我们综合对比多个模型的性能！

In [ ]:
# 练习 3.6: 模型对比

def compare_uplift_models(
    y_true: np.ndarray,
    treatment: np.ndarray,
    model_predictions: dict  # {'model_name': uplift_scores}
) -> pd.DataFrame:
    """
    对比多个 Uplift 模型的性能
    
    TODO: 实现多模型对比
    
    Args:
        y_true: 真实结果
        treatment: 处理状态
        model_predictions: 字典，key 为模型名，value 为预测 uplift
    
    Returns:
        DataFrame with comparison metrics
    """
    results = []
    
    for model_name, scores in model_predictions.items():
        # TODO: 计算各项指标
        auuc = calculate_auuc(y_true, treatment, scores)
        
        # Top 10% uplift
        n = len(y_true)
        order = np.argsort(scores)[::-1]
        top_10_idx = order[:int(0.1 * n)]
        
        y_top = y_true[top_10_idx]
        t_top = treatment[top_10_idx]
        
        if (t_top == 1).sum() > 0 and (t_top == 0).sum() > 0:
            top_10_uplift = y_top[t_top == 1].mean() - y_top[t_top == 0].mean()
        else:
            top_10_uplift = 0
        
        results.append({
            'Model': model_name,
            'AUUC': auuc,
            'Top 10% Uplift': top_10_uplift
        })
    
    return pd.DataFrame(results)

# 测试
comparison = compare_uplift_models(
    Y, T,
    {
        'Good Model': model1_pred,
        'Poor Model': model2_pred,
        'Random': np.random.randn(n)
    }
)
if comparison is not None and len(comparison) > 0:
    print("模型对比结果:")
    print(comparison.to_string(index=False))
else:
    print("[未完成] 请完成 compare_uplift_models 函数")

---

## 思考题

### 1. Qini 曲线和 ROC 曲线有什么相似和不同之处?

**你的答案:**

相似:
-

不同:
-

### 2. 为什么 Qini 曲线需要调整控制组人数 (n_t / n_c 的权重)?

**你的答案:**


### 3. AUUC 的值越大越好吗? AUUC 的取值范围是什么?

**你的答案:**


### 4. 如果某个模型的 Qini 曲线在某些区域低于随机基线，这意味着什么?

**你的答案:**


### 5. Top 10% Uplift 指标有什么实际意义? 什么时候应该关注这个指标?

**你的答案:**


### 6. 在确定最优干预比例时，除了 ROI，还应该考虑哪些因素?

**你的答案:**
-
-
-

### 7. 如果没有真实的 uplift (只有观测数据 Y, T)，如何评估 Uplift 模型?

**你的答案:**


---

## 总结

| 评估方法 | 用途 | 优点 | 注意事项 |
|---------|------|------|--------|
| **Qini 曲线** | 评估排序能力 | 直观，类似 ROC | 需要处理/控制组人数调整 |
| **AUUC** | 单一数值指标 | 便于模型对比 | 受样本量影响 |
| **Uplift by Decile** | 分桶验证 | 业务可解释 | 需要足够样本 |
| **累积增益** | 绝对价值评估 | 直接与业务挂钩 | 依赖成本收益假设 |
| **最优干预比例** | 决策支持 | 实用性强 | 假设边际效应可加 |

### 恭喜你完成了 Chapter 3 的所有练习！

下一章我们将学习 **深度因果模型** —— TARNet、DragonNet 等神经网络方法！

---

*恭喜你完成了 Uplift 模型评估的学习！* 🎉